In [1]:
import pandas as pd
import os
from pyDOE import *
from scipy.io import netcdf as nc

### Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl '+data_url+' > params.csv'
os.system(cmd)

0

### Read in csv data, filtering by the "include" column

In [3]:
data     = pd.read_csv('params.csv')
included = data['include']==1
params   = data.loc[included,['name','min','max','dim2','location']]
params

,name,min,max,dim2,location
1,displar,4.000000e-01,9.500000e-01,NaN,P
2,dleaf,8.100000e-03,2.430000e-01,NaN,P
4,kmax,1.000000e-08,3.000000e-08,4,P
11,baseflow_scalar,5.000000e-04,1.000000e-01,NaN,N
12,maximum_leaf_wetted_fraction,1.000000e-02,5.000000e-01,NaN,N


### Generate parameter sampling
 * option available for latin hypercube (LHC) or one-at-a-time (OAAT)
 * careful, each time you run LHC you get a new random draw

In [31]:
sampling_protocol = 'OAAT'
prefix = sampling_protocol
nparam = len(params['name'])  #number of parameters

if sampling_protocol == 'LHC':
    #define sample size (number of ensemble members)
    nsamp = 10

    # Generate the latin hypercube sample
    lhd = lhs(nparam, samples=int(nsamp))

    # scale according to parameter range
    param_array = ((params['max'] - params['min']).values)*lhd + params['min'].values

elif sampling_protocol == 'OAAT':
    #nan is code for keep the default value
    nsamp = 2*nparam
    param_array = np.nan*np.ones([nsamp,nparam])
    mins_index = (np.arange(0,nsamp,2),np.arange(0,nparam,1))
    maxs_index = (np.arange(1,nsamp,2),np.arange(0,nparam,1))
    param_array[mins_index]=params['min']
    param_array[maxs_index]=params['max']

# store in a pandas dataframe
psets = pd.DataFrame(data=param_array, index=None, columns=params['name'])
psets

name,displar,dleaf,kmax,baseflow_scalar,maximum_leaf_wetted_fraction
0,0.40,NaN,NaN,NaN,NaN
1,0.95,NaN,NaN,NaN,NaN
2,NaN,0.0081,NaN,NaN,NaN
3,NaN,0.2430,NaN,NaN,NaN
4,NaN,NaN,1.000000e-08,NaN,NaN
5,NaN,NaN,3.000000e-08,NaN,NaN
6,NaN,NaN,NaN,0.0005,NaN
7,NaN,NaN,NaN,0.1000,NaN
8,NaN,NaN,NaN,NaN,0.01
9,NaN,NaN,NaN,NaN,0.50


## Generate parameter files
* ### this will overwrite parameter files!!
* ### proceed with caution

In [32]:
#create all the new parameter files, copied from basepftfile
# 0th case is the default parameter set
basepftfile = "../basecase/clm5_params.c171117.nc"
for i in range(nsamp+1):
    targetpftfile = "../paramfiles/"+prefix+str(i).zfill(4)+".nc"
    print(targetpftfile)
    os.system('cp '+basepftfile+' '+targetpftfile)
    
#modify the parameter values
for name,loc in zip(params['name'],params['location']):
    if loc=='P':
        print(name)
        for i in range(nsamp):
            if ~np.isnan(psets[name][i]):
                pftfile = "../paramfiles/"+prefix+str(i+1).zfill(4)+".nc"
                f = nc.netcdf_file(pftfile, 'a') # could probably update this to use xarray
                var = f.variables[name]
                var[:] = psets[name][i] 
                # TODO: pft variations
                f.close()

../paramfiles/OAAT0000.nc
../paramfiles/OAAT0001.nc
../paramfiles/OAAT0002.nc
../paramfiles/OAAT0003.nc
../paramfiles/OAAT0004.nc
../paramfiles/OAAT0005.nc
../paramfiles/OAAT0006.nc
../paramfiles/OAAT0007.nc
../paramfiles/OAAT0008.nc
../paramfiles/OAAT0009.nc
../paramfiles/OAAT0010.nc
displar
dleaf
kmax


### Generate namelist files

First create files and specify the parameter file

In [33]:
#create the namelist mod files
for i in range(nsamp+1):
    nlfile = "../namelist_mods/"+prefix+str(i).zfill(4)+".txt" 
    with open(nlfile,"w") as file:
        output = "! user_nl_clm namelist options written by generate_params:\n"
        file.write(output)
#populate with mods
for name,loc in zip(params['name'],params['location']):
    if loc=='N':
        for i in range(nsamp):
            if ~np.isnan(psets[name][i]):
                nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt"  
                with open(nlfile,"a") as file: # key is using "a" for append option
                    output = "%s=%s\n" % (name, psets[name][i]) #round??
                    file.write(output) 

Then populate them with parameter values\
I couldn't do this in the same loop as creating them because it was overriding content when iterating over parameters, but there may be a better way